In [35]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import load_dataset, DatasetDict, Dataset

poly_ai = load_dataset("PolyAI/minds14", "en-US")

poly_ai

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 563
    })
})

In [6]:
dataset_to_use = DatasetDict()

dataset_to_use["train"] = Dataset.from_dict(poly_ai["train"][0:450])
dataset_to_use["test"] = Dataset.from_dict(poly_ai["train"][450:])


In [7]:
dataset_to_use

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 450
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 113
    })
})

In [8]:
dataset_to_use["train"][0]["audio"]

{'array': [0.0,
  0.000244140625,
  -0.000244140625,
  0.0,
  -0.000244140625,
  0.0,
  0.000244140625,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.000244140625,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.000244140625,
  -0.000244140625,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.000244140625,
  0.0,
  -0.000244140625,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.000244140625,
  0.0,
  0.0,
  0.0,
  -0.000244140625,
  0.0,
  0.0,
  -0.000244140625,
  0.0,
  -0.000244140625,
  0.0,
  0.0,
  0.0,
  0.0,
  0.000244140625,
  0.000244140625,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.000244140625,
  0.0,
  0.0,
  0.0,
  0.000244140625,
  0.0,
  0.0,
  0.000244140625,
  -0.000244140625,
  0.0,
  0.0,
  0.000244140625,
  0.0,
  0.0,
  0.000244140625,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.000244140625,
  0.000244140625,
  0.0,
  0.0,
  0.0,
  0.0,
  0.000

In [9]:
from transformers.models.whisper.tokenization_whisper import TO_LANGUAGE_CODE
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small", language="english", task="transcribe"
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
dataset_to_use = dataset_to_use.cast_column("audio",Audio(sampling_rate=sampling_rate))

In [11]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["english_transcription"],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

In [12]:
prepare_dataset(dataset_to_use["train"][0])

{'input_features': array([[[-0.67151165, -0.62832355, -0.67151165, ..., -0.67151165,
         -0.67151165, -0.67151165],
        [-0.67151165, -0.6334348 , -0.67151165, ..., -0.67151165,
         -0.67151165, -0.67151165],
        [-0.67151165, -0.67151165, -0.67151165, ..., -0.67151165,
         -0.67151165, -0.67151165],
        ...,
        [-0.67151165, -0.67151165, -0.67151165, ..., -0.67151165,
         -0.67151165, -0.67151165],
        [-0.67151165, -0.67151165, -0.67151165, ..., -0.67151165,
         -0.67151165, -0.67151165],
        [-0.67151165, -0.67151165, -0.67151165, ..., -0.67151165,
         -0.67151165, -0.67151165]]], dtype=float32), 'labels': [50258, 50259, 50359, 50363, 40, 576, 411, 281, 992, 493, 257, 7225, 2696, 365, 452, 4975, 50257], 'input_length': 10.837375}

In [13]:
dataset_to_use = dataset_to_use.map(prepare_dataset,remove_columns=dataset_to_use.column_names["train"], num_proc = 1)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

In [14]:
dataset_to_use

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 450
    })
    test: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 113
    })
})

In [15]:
max_input_length = 30.0


def is_audio_in_length_range(length):
    return length < max_input_length

dataset_to_use["tain"] = dataset_to_use["train"].filter(is_audio_in_length_range, input_columns = ['input_length'])

Filter:   0%|          | 0/450 [00:00<?, ? examples/s]

In [16]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [17]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor = processor)

In [18]:
!pip install jiwer

In [19]:
import evaluate

metric = evaluate.load("wer")

In [20]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [21]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [22]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="english", task="transcribe", use_cache=True
)

In [23]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-dv",  # name on the HF Hub
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=4000,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/training_args.py:1504: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset_to_use["train"],
    eval_dataset=dataset_to_use["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

max_steps is given, it will override any value given in num_train_epochs


In [25]:
trainer.train()


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer Ortho,Wer
500,0.000100,0.602929,0.254781,0.257969
1000,0.000000,0.656808,0.247995,0.253247
1500,0.000000,0.690974,0.249846,0.255608
2000,0.000000,0.717311,0.247995,0.253837
2500,0.000000,0.740206,0.248612,0.254427
3000,0.000000,0.762435,0.256632,0.262102
3500,0.000000,0.779991,0.264035,0.269185
4000,0.000000,0.796162,0.262184,0.267414


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 65

TrainOutput(global_step=4000, training_loss=0.03593853246121944, metrics={'train_runtime': 9744.4921, 'train_samples_per_second': 6.568, 'train_steps_per_second': 0.41, 'total_flos': 1.791595882266624e+19, 'train_loss': 0.03593853246121944, 'epoch': 137.93103448275863})

In [39]:
kwargs = {
    # "dataset_tags": "PolyAI/minds14",
    "dataset":"minds14",
    "finetuned_from": "openai/whisper-tiny",
    "model_name": "openai/whisper-small-finetuned-gtzan",
    "tasks": "automatic-speech-recognition",
}

In [40]:
trainer.push_to_hub(**kwargs)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


CommitInfo(commit_url='https://huggingface.co/avnishkanungo/whisper-small-dv/commit/d3c277ee647a5de21d314d2e9dc819b71650b687', commit_message='End of training', commit_description='', oid='d3c277ee647a5de21d314d2e9dc819b71650b687', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
from transformers import pipeline

model_id = "avnishkanungo/whisper-small-dv"  # update with your model id
pipe = pipeline("automatic-speech-recognition", model=model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
def transcribe_speech(filepath):
    output = pipe(
        filepath,
        max_new_tokens=256,
        generate_kwargs={
            "task": "transcribe",
            "language": "english",
        },  # update with the language you've fine-tuned on
        chunk_length_s=30,
        batch_size=8,
    )
    return output["text"]

In [5]:
transcribe_speech("/teamspace/studios/this_studio/harvard.wav")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'the stale smell of old beer lingers it takes heat to bring out the odor a cold dip restores health and zest a salt pickle tastes fine with ham tacos all pastore are my favorite a zestful food is the hot cross bun'

In [6]:
!pip install sounddevice

In [ ]:
transcribe_speech(audio_data)

In [8]:
!sudo apt-get install libportaudio2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 3 not upgraded.
Need to get 65.4 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudio2 amd64 19.6.0-1build1 [65.4 kB]
Fetched 65.4 kB in 0s (1073 kB/s)        
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 64811 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1build1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1build1) ...
Setting up libportaudio2:amd64 (19.6.0-1build1) ...
Processing triggers for libc-bin (2.31-0ubuntu9.16) ...


In [4]:
!sudo apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libass9 libasyncns0 libavc1394-0 libavdevice58 libavfilter7 libavresample4
  libbs2b0 libcaca0 libcdio-cdda2 libcdio-paranoia2 libcdio18 libdc1394-22
  libfftw3-double3 libflac8 libflite1 libiec61883-0 libjack-jackd2-0
  liblilv-0-0 libmysofa1 libnorm1 libopenal-data libopenal1 libpgm-5.2-0
  libpostproc55 libpulse0 libraw1394-11 librubberband2 libsamplerate0
  libsdl2-2.0-0 libserd-0-0 libslang2 libsndfile1 libsndio7.0 libsodium23
  libsord-0-0 libsratom-0-0 libusb-1.0-0 libvidstab1.1 libxss1 libxv1 libzmq5
Suggested packages:
  ffmpeg-doc libfftw3-bin libfftw3-dev jackd2 libportaudio2 pulseaudio
  libraw1394-doc serdi sndiod sordi
The following NEW packages will be installed:
  ffmpeg libass9 libasyncns0 libavc1394-0 libavdevice58 libavfilter7
  libavresample4 libbs2b0 libcaca0 libcdio-cdda2 libcdio-paranoia2 libcdio18
  libdc1394-22 l

In [50]:
import gradio as gr

demo = gr.Blocks()

mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="microphone", type="filepath"),
    outputs=gr.components.Textbox(),
)

file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="upload", type="filepath"),
    outputs=gr.components.Textbox(),
)

In [51]:
with demo:
    gr.TabbedInterface(
        [mic_transcribe, file_transcribe],
        ["Transcribe Microphone", "Transcribe Audio File"],
    )

demo.launch(debug=True)

ImportError: cannot import name 'http_server' from 'gradio' (/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/gradio/__init__.py)